In [1]:
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
import SimpleITK as sitk
import tensorflow as tf
from tensorflow import keras
from keras.layers import AveragePooling3D
from multiprocessing import Process, Manager

import time
import copy
from sklearn.datasets import load_digits
from sklearn.cluster import KMeans
from sklearn.metrics import normalized_mutual_info_score
import os
import imageio
import pydicom as dicom
import pandas as pd
import sys

import SimpleITK
from scipy import ndimage as ndi
from skimage.segmentation import clear_border
from skimage.measure import label, regionprops
from skimage.morphology import disk, dilation, binary_erosion, binary_closing
from skimage.filters import roberts, sobel
import cv2
from KEM_EXPE import *
from experiment_auxiliary import *
import csv

sys.path.append('../')
sys.path.append('../Models/')
from utils import *
from GMM import *
from Kmeans import *
%matplotlib inline

In [2]:
path = '/database/datasets/Classics/LIDCdata/'  # LIDC dataset
patient_folders = os.listdir(path)

In [3]:
patient_folders

['LIDC-IDRI-0194',
 'LIDC-IDRI-0273',
 'LIDC-IDRI-0405',
 'LIDC-IDRI-0250',
 'LIDC-IDRI-0167',
 'LIDC-IDRI-0205',
 'LIDC-IDRI-0137',
 'LIDC-IDRI-0163',
 'LIDC-IDRI-0166',
 'LIDC-IDRI-0994',
 'LIDC-IDRI-0165',
 'LIDC-IDRI-0232',
 'LIDC-IDRI-0168',
 'LIDC-IDRI-0248',
 'LIDC-IDRI-0385',
 'LIDC-IDRI-0559',
 'LIDC-IDRI-0290',
 'LIDC-IDRI-0939',
 'LIDC-IDRI-0247',
 'LIDC-IDRI-0811',
 'LIDC-IDRI-0543',
 'LIDC-IDRI-0149',
 'LIDC-IDRI-0766',
 'LIDC-IDRI-0180',
 'LIDC-IDRI-0610',
 'LIDC-IDRI-0072',
 'LIDC-IDRI-0230',
 'LIDC-IDRI-0176',
 'LIDC-IDRI-1010',
 'LIDC-IDRI-0200',
 'LIDC-IDRI-0251',
 'LIDC-IDRI-0261',
 'LIDC-IDRI-0267',
 'LIDC-IDRI-0174',
 'LIDC-IDRI-0399',
 'LIDC-IDRI-0191',
 'LIDC-IDRI-0237',
 'LIDC-IDRI-0258',
 'LIDC-IDRI-0377',
 'LIDC-IDRI-0260',
 'LIDC-IDRI-0282',
 'LIDC-IDRI-0088',
 'LIDC-IDRI-0239',
 'LIDC-IDRI-0233',
 'LIDC-IDRI-1002',
 'LIDC-IDRI-0266',
 'LIDC-IDRI-0186',
 'LIDC-IDRI-0875',
 'LIDC-IDRI-0470',
 'LIDC-IDRI-0203',
 'LIDC-IDRI-0276',
 'LIDC-IDRI-0279',
 'LIDC-IDRI-

In [4]:
len(patient_folders)

158

In [5]:
K = 3
index_list = np.arange(len(patient_folders))
lung_threshold = -200
training_ratio = 0.8
Ch = 0.2217
kernel_shape = (3, 3, 3)

In [6]:
to_csv_path = "./experiment_SPE.csv"

# with open(to_csv_path, 'w', newline='', encoding='utf-8') as f:
#     csv_write = csv.writer(f)  
#     csv_write.writerow([f"index", "path", "concentrate_lung_time", 
#                         "kem_spe", "kmeans_spe", "gmm_spe"])

In [7]:
stats_dict = {'K': K,
              'lung_threshold': lung_threshold,
              'training_ratio': training_ratio,
              'Ch': Ch,
              'kernel_shape': kernel_shape,
              'to_csv_path': to_csv_path,
              'index': None,
              }
# transform the dict so that it can be used in the following Process
manager = Manager()
stats_dict = manager.dict(stats_dict)

In [8]:
for index in index_list:
    if patient_folders[index][0] == '.':
        continue
    print(index)
    np.random.seed(index)
    tf.random.set_seed(index)
    stats_dict['index'] = index
    process_eval = Process(target=experiment_compare_SPE, args=(stats_dict,))
    process_eval.start()
    process_eval.join()